# Nettoyage du jeu de données

## 1. Suppression des doublons
L'objectif de ce notebook est d'écarter les images en doublons issues de notre dataset

In [1]:
import os
import pandas as pd
import numpy as np
import glob
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import hashlib
import cv2

In [2]:
# Chemin du répertoire contenant les images
repertoire = r'D:\aprojet\fev24_cds_plants\data\color'


In [3]:
# Chargement du dataframe depuis le fichier CSV
df = pd.read_csv('dataset_plantdisease.csv')

In [4]:
# Analyse des données
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54303 entries, 0 to 54302
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Chemin          54303 non-null  object
 1   Classe          54303 non-null  object
 2   Nom_image       54303 non-null  object
 3   Largeur         54303 non-null  int64 
 4   Hauteur         54303 non-null  int64 
 5   hash            54303 non-null  object
 6   format          54303 non-null  object
 7   mode            54303 non-null  object
 8   Plante          54303 non-null  object
 9   Maladie/Statut  54303 non-null  object
dtypes: int64(2), object(8)
memory usage: 4.1+ MB


In [5]:
duplicates = df.groupby('hash').filter(lambda x: len(x) > 1)
duplicates.reset_index(drop=True, inplace=True)
pd.set_option('display.max_colwidth', None)
duplicates_sorted = duplicates.sort_values(by='hash')

# Réinitialisation de l'index si nécessaire
duplicates_sorted.reset_index(drop=True, inplace=True)

# Affichage des doublons triés par hash MD5
print("Nombre de doublons trouvés :", len(duplicates_sorted))
pd.set_option('display.max_colwidth', None)
display(duplicates_sorted)
duplicates_sorted.info()

Nombre de doublons trouvés : 42


,Chemin,Classe,Nom_image,Largeur,Hauteur,hash,format,mode,Plante,Maladie/Statut
0,D:\aprojet\fev24_cds_plants\data\color\Tomato___healthy\068e324c-faf6-40d6-8f83-578907f1cac5___GH_HL Leaf 466.1.JPG,Tomato___healthy,068e324c-faf6-40d6-8f83-578907f1cac5___GH_HL Leaf 466.1.JPG,256,256,0b29a96c6ddbfa17b804fe09d7b5c5aa,JPEG,RGB,Tomato,healthy
1,D:\aprojet\fev24_cds_plants\data\color\Tomato___healthy\34c81c57-e1fa-49dd-a49d-34fe8b2385fe___GH_HL Leaf 466.1.JPG,Tomato___healthy,34c81c57-e1fa-49dd-a49d-34fe8b2385fe___GH_HL Leaf 466.1.JPG,256,256,0b29a96c6ddbfa17b804fe09d7b5c5aa,JPEG,RGB,Tomato,healthy
2,D:\aprojet\fev24_cds_plants\data\color\Tomato___Late_blight\bd4f09bd-ee85-4ab1-bce0-8cde3fdd7f1b___GHLB_PS Leaf 23.7 Day 13.jpg,Tomato___Late_blight,bd4f09bd-ee85-4ab1-bce0-8cde3fdd7f1b___GHLB_PS Leaf 23.7 Day 13.jpg,256,256,0ce1c770837fd4cd35cc0b7901da6a57,JPEG,RGB,Tomato,Late_blight
3,D:\aprojet\fev24_cds_plants\data\color\Tomato___Late_blight\48c55974-9fe9-4f4b-94f7-c8cd127d1e05___GHLB_PS Leaf 23.7 Day 13.jpg,Tomato___Late_blight,48c55974-9fe9-4f4b-94f7-c8cd127d1e05___GHLB_PS Leaf 23.7 Day 13.jpg,256,256,0ce1c770837fd4cd35cc0b7901da6a57,JPEG,RGB,Tomato,Late_blight
4,D:\aprojet\fev24_cds_plants\data\color\Apple___healthy\5192db55-4aa7-421c-92d4-c2dac79e7379___RS_HL 6273.JPG,Apple___healthy,5192db55-4aa7-421c-92d4-c2dac79e7379___RS_HL 6273.JPG,256,256,1987c933d1b47a1ef89727337ffdcc83,JPEG,RGB,Apple,healthy
5,D:\aprojet\fev24_cds_plants\data\color\Apple___healthy\9b75de13-d4b0-4b3f-988c-3e9926eef957___RS_HL 6273.JPG,Apple___healthy,9b75de13-d4b0-4b3f-988c-3e9926eef957___RS_HL 6273.JPG,256,256,1987c933d1b47a1ef89727337ffdcc83,JPEG,RGB,Apple,healthy
6,D:\aprojet\fev24_cds_plants\data\color\Apple___healthy\1ab5e019-e5f0-4d8e-a252-94cb0aab8b0a___RS_HL 6269.JPG,Apple___healthy,1ab5e019-e5f0-4d8e-a252-94cb0aab8b0a___RS_HL 6269.JPG,256,256,1e86760b7721d066126b20056e8d5bb2,JPEG,RGB,Apple,healthy
7,D:\aprojet\fev24_cds_plants\data\color\Apple___healthy\3673d121-b5de-481c-b057-d4ee5b4959b1___RS_HL 6269.JPG,Apple___healthy,3673d121-b5de-481c-b057-d4ee5b4959b1___RS_HL 6269.JPG,256,256,1e86760b7721d066126b20056e8d5bb2,JPEG,RGB,Apple,healthy
8,D:\aprojet\fev24_cds_plants\data\color\Tomato___Late_blight\d81682aa-746b-4e07-af2b-52ebb6f4c017___GHLB2 Leaf 102.JPG,Tomato___Late_blight,d81682aa-746b-4e07-af2b-52ebb6f4c017___GHLB2 Leaf 102.JPG,256,256,288508d7883f96452a46745acd3d00d1,JPEG,RGB,Tomato,Late_blight
9,D:\aprojet\fev24_cds_plants\data\color\Tomato___Late_blight\d6e6897a-5083-4914-9903-804c5684a956___GHLB2 Leaf 102.JPG,Tomato___Late_blight,d6e6897a-5083-4914-9903-804c5684a956___GHLB2 Leaf 102.JPG,256,256,288508d7883f96452a46745acd3d00d1,JPEG,RGB,Tomato,Late_blight


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Chemin          42 non-null     object
 1   Classe          42 non-null     object
 2   Nom_image       42 non-null     object
 3   Largeur         42 non-null     int64 
 4   Hauteur         42 non-null     int64 
 5   hash            42 non-null     object
 6   format          42 non-null     object
 7   mode            42 non-null     object
 8   Plante          42 non-null     object
 9   Maladie/Statut  42 non-null     object
dtypes: int64(2), object(8)
memory usage: 3.4+ KB


In [11]:
doublons = df[df.duplicated(subset=['hash'], keep=False)]

# Sélectionner un représentant unique pour chaque groupe de doublons
representants = doublons.drop_duplicates(subset=['hash'])

# Créer le nouveau dataframe avec les représentants sélectionnés
nouveau_df = pd.DataFrame({
    'metric': 'doublon',
    'class': representants['Classe'],
    'filename': representants['Nom_image'],
    'comment': 'doublon'
})


nouveau_df.to_csv('files_to_delete.csv',sep=';', index=False)